# Create a Hushh Vibe Catalog

Download an example product catalog from this [Kaggle dataset](https://www.kaggle.com/datasets/paramaggarwal/fashion-product-images-small).

Unzip it into the `notebooks` directory and it will be ignored by git.


In [6]:
!kaggle datasets download paramaggarwal/fashion-product-images-small --force 

100%|███████████████████████████████████████▉| 565M/565M [00:51<00:00, 11.3MB/s]
100%|████████████████████████████████████████| 565M/565M [00:51<00:00, 11.6MB/s]


In [7]:
! unzip -o fashion-product-images-small.zip > /dev/null

Take a look at the images

In [20]:
import glob
print(glob.glob("images/*")[:10])

['images/9733.jpg', 'images/14147.jpg', 'images/52112.jpg', 'images/6400.jpg', 'images/34297.jpg', 'images/24084.jpg', 'images/12536.jpg', 'images/54563.jpg', 'images/15259.jpg', 'images/35189.jpg']


In [21]:
import glob
print(glob.glob("myntradataset/images/*")[:10])

['myntradataset/images/9733.jpg', 'myntradataset/images/14147.jpg', 'myntradataset/images/52112.jpg', 'myntradataset/images/6400.jpg', 'myntradataset/images/34297.jpg', 'myntradataset/images/24084.jpg', 'myntradataset/images/12536.jpg', 'myntradataset/images/54563.jpg', 'myntradataset/images/15259.jpg', 'myntradataset/images/35189.jpg']


Import the necessary classes.

In [23]:
from hushh import Catalog, Product

ImportError: cannot import name 'Catalog' from 'hushh' (/opt/homebrew/Caskroom/miniconda/base/envs/hushh-vibe-catalog-reader/lib/python3.11/site-packages/hushh/__init__.py)